In [1]:
import pandas as pd
import numpy as np

# 훈련 데이터 전처리

In [2]:
data = pd.read_excel('C:\\Users\\Playdata\\Workspace\\Mini_proj\\MachineLearning\\Resource\\Data_Train.xlsx')
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [3]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [ ]:
df.Date_of_Journey = pd.to_datetime(df.Date_of_Journey)
df.info()

In [5]:
df.drop(columns=['Route'],inplace=True)

In [6]:
df['Dep_Time'] = pd.to_datetime(df['Dep_Time'], format= '%H:%M').dt.time
df['Duration_hour'] = df.Duration.str.extract('(\d+)h')
df['Duration_min'] = df.Duration.str.extract('(\d+)m').fillna(0)
df.drop('Duration', axis=1, inplace=True)
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Total_Stops,Additional_Info,Price,Duration_hour,Duration_min
0,IndiGo,2019-03-24,Banglore,New Delhi,22:20:00,01:10 22 Mar,non-stop,No info,3897,2,50
1,Air India,2019-01-05,Kolkata,Banglore,05:50:00,13:15,2 stops,No info,7662,7,25
2,Jet Airways,2019-09-06,Delhi,Cochin,09:25:00,04:25 10 Jun,2 stops,No info,13882,19,0
3,IndiGo,2019-12-05,Kolkata,Banglore,18:05:00,23:30,1 stop,No info,6218,5,25
4,IndiGo,2019-01-03,Banglore,New Delhi,16:50:00,21:35,1 stop,No info,13302,4,45


# Duration 에서 이상치 drop

In [7]:
df[df['Duration_hour'].isna()]
df.drop(index=6474,inplace=True)
len(df)

10682

In [8]:
# str -> int64 로 변경
df.Duration_hour = df.Duration_hour.astype('int64')
df.Duration_min = df.Duration_min.astype('int64')

In [9]:
# Duration_hour을 분으로 변경
df.Duration_hour = df.Duration_hour*60
k = df.Duration_hour+df.Duration_min
k.head()

0     170
1     445
2    1140
3     325
4     285
dtype: int64

# Airline 전처리

In [25]:
# count값에 맞춰 index 넘버링하기
air_count = df.Airline.value_counts().index

# 200 보다 적은 수의 airline은 Others 로 변환
airlist = [l for l in air_count if list(df.Airline).count(l) < 200]
df.Airline = df.Airline.replace(airlist, 'Others')
df.Airline.value_counts()

# 컬럼 미리 생성
df['Air_col'] = 0

df.Airline.value_counts()

Jet Airways          3849
IndiGo               2053
Air India            1751
Multiple carriers    1196
SpiceJet              818
Vistara               479
Air Asia              319
Others                217
Name: Airline, dtype: int64

In [26]:
# Air_col : Airline을 번호로 분류
for t in range(len(air_count)):
    df.loc[df.Airline == air_count[t], 'Air_col'] = t

print(df.Air_col.value_counts())
print(df.head())

0    4066
1    2053
2    1751
3    1196
4     818
5     479
6     319
Name: Air_col, dtype: int64
       Airline Date_of_Journey    Source Destination  Dep_Time  Arrival_Time  \
0       IndiGo      2019-03-24  Banglore   New Delhi  22:20:00  01:10 22 Mar   
1    Air India      2019-01-05   Kolkata    Banglore  05:50:00         13:15   
2  Jet Airways      2019-09-06     Delhi      Cochin  09:25:00  04:25 10 Jun   
3       IndiGo      2019-12-05   Kolkata    Banglore  18:05:00         23:30   
4       IndiGo      2019-01-03  Banglore   New Delhi  16:50:00         21:35   

  Total_Stops Additional_Info  Price Duration_hour Duration_min  Air_col  
0    non-stop         No info   3897             2           50        1  
1     2 stops         No info   7662             7           25        2  
2     2 stops         No info  13882            19            0        0  
3      1 stop         No info   6218             5           25        1  
4      1 stop         No info  13302          

# 출발, 도착 시간 전처리

In [28]:
## error

df.Dep_Time.str.extract('([0-9]+)\:')
df.Arrival_Time.str.extract('([0-9]+)\:')



AttributeError: Can only use .str accessor with string values!